----------------------------------------

Copyright 2018 Google LLC 

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

[http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and limitations under the License.

----------------------------------------

# Energy Forecasting with AutoML Tables

To use this Colab notebook, copy it to your own Google Drive and open it with [Colaboratory](https://colab.research.google.com/) (or Colab). To run a cell hold the Shift key and press the Enter key (or Return key). Colab automatically displays the return value of the last line in each cell. Refer to [this page](https://colab.research.google.com/notebooks/welcome.ipynb) for more information on Colab.

You can run a Colab notebook on a hosted runtime in the Cloud. The hosted VM times out after 90 minutes of inactivity and you will lose all the data stored in the memory including your authentication data. If your session gets disconnected (for example, because you closed your laptop) for less than the 90 minute inactivity timeout limit, press 'RECONNECT' on the top right corner of your notebook and resume the session. After Colab timeout, you'll need to

1.   Re-run the initialization and authentication.
2.   Continue from where you left off. You may need to copy-paste the value of some variables such as the `dataset_name` from the printed output of the previous cells.

Alternatively you can connect your Colab notebook to a [local runtime](https://research.google.com/colaboratory/local-runtimes.html).

## 1. Project set up





Follow the [AutoML Tables documentation](https://cloud.google.com/automl-tables/docs/) to
* Create a Google Cloud Platform (GCP) project.
* Enable billing.
* Apply to whitelist your project.
* Enable AutoML API.
* Enable AutoML Talbes API.
* Create a service account, grant required permissions, and download the service account private key.

You also need to upload your data into Google Cloud Storage (GCS) or BigQuery. For example, to use GCS as your data source
* Create a GCS bucket.
* Upload the training and batch prediction files.


**Warning:** Private keys must be kept secret. If you expose your private key it is recommended to revoke it immediately from the Google Cloud Console.



---



## 2. Initialize and authenticate
This section runs intialization and authentication. It creates an authenticated session which is required for running any of the following sections.

### Install the client library
Run the following cell to install the client library using `pip`.

In [ ]:
#@title Install AutoML Tables client library { vertical-output: true }

!pip install google-cloud-automl

### Set Project and Location

Enter your GCP project ID.

In [ ]:
#@title GCP project ID and location

project_id = 'energy-forecasting' #@param {type:'string'}
location = 'us-central1' #@param {type:'string'}

### Authenticate using service account key
Run the following cell. Click on the 'Choose Files' button and select the service account private key file. If your Service Account key file or folder is hidden, you can reveal it in a Mac by pressing the <b>Command + Shift + .</b> combo.

In [ ]:
#@title Authenticate using service account key and create a client. { vertical-output: true }

from google.oauth2 import service_account
from google.colab import files
from google.cloud import automl_v1beta1

# Upload service account key
keyfile_upload = files.upload()
keyfile_name = list(keyfile_upload.keys())[0]
# Authenticate and create an AutoML client.
credentials = service_account.Credentials.from_service_account_file(keyfile_name)
client = automl.TablesClient(project=project_id, region=location, credentials=credentials)



---



## 3. Import training data

### Create dataset

Select a dataset display name and pass your table source information to create a new dataset.

In [ ]:
#@title Create dataset { vertical-output: true, output-height: 200 }

dataset_display_name = 'energy_forcasting_solution' 
dataset = client.create_dataset(dataset_display_name)
print(dataset.name) # unique to this dataset
dataset

### Import data

You can import your data to AutoML Tables from GCS or BigQuery. You can create a GCS bucket and upload the  data into your bucket. The URI for your file is `gs://BUCKET_NAME/FOLDER_NAME1/FOLDER_NAME2/.../FILE_NAME`. Alternatively you can create a BigQuery table and upload the data into the table. The URI for your table is `bq://PROJECT_ID.DATASET_ID.TABLE_ID`.

Importing data may take a few minutes or hours depending on the size of your data. __If your Colab times out__, run the following command to retrieve your dataset. Replace `dataset_name` with its actual value obtained in the preceding cells.

```python
    # This will work if your display name ('energy_forecasting_solution') is unique to your project.
    dataset = client.get_dataset(dataset_display_name=dataset_display_name)
    # OR, if you have multiple datasets with the same display name ('energy_forecasting_solution'), use the
    # unique indentifier acquired from the above cell ( print(dataset.name) ).
    dataset = client.get_dataset(dataset_name=dataset_name)
```

In [ ]:
 #@title Import data { vertical-output: true }

dataset_bq_input_uri = 'bq://energy-forecasting.Energy.automldata' #@param {type: 'string'}

import_data_operation = client.import_data(
    dataset=dataset,
    bigquery_input_uri=dataset_bq_input_uri
)

print('Dataset import operation: {}'.format(import_data_response.operation))

# Wait until import is done.
import_data_result = import_data_response.result()
import_data_result

### Review the specs

Run the following command to see table specs such as row count.

In [ ]:
#@title Table schema { vertical-output: true }

# List table specs
list_table_specs_response = client.list_table_specs(dataset=dataset)
table_specs = [s for s in list_table_specs_response]

# List column specs
list_column_specs_response = client.list_column_specs(dataset=dataset)
column_specs = {s.display_name: s for s in list_column_specs_response}

# Print Features and data_type:

features = [(key, data_types.TypeCode.Name(value.data_type.type_code)) for key, value in column_specs.items()]
print('Feature list:\n')
for feature in features:
    print(feature[0],':', feature[1])

___

## 4. Update dataset: assign a label column and enable nullable columns

AutoML Tables automatically detects your data column type. For example, for the [Iris dataset](https://storage.cloud.google.com/rostam-193618-tutorial/automl-tables-v1beta1/iris.csv) it detects `species` to be categorical and `petal_length`, `petal_width`, `sepal_length`, and `sepal_width` to be numerical. Depending on the type of your label column, AutoML Tables chooses to run a classification or regression model. If your label column contains only numerical values, but they represent categories, change your label column type to categorical by updating your schema.

### Update a column: set as categorical

In [ ]:
#@title Update dataset { vertical-output: true }

column_to_category = 'hour' #@param {type: 'string'}

update_column_response = client.update_column_spec(
    dataset=dataset,
    column_spec_display_name=column_to_category,
    type_code='CATEGORY'
)

update_column_response.display_name, update_column_response.data_type 

### Update dataset: assign a target and split column

In [ ]:
#@title Update dataset { vertical-output: true }

target_column_name = 'price' #@param {type: 'string'}
split_column_name = 'split' #@param {type: 'string'}

client.set_target_column(
    dataset=dataset,
    column_spec_display_name=target_column_name,
)

client.set_test_train_column(
    dataset=dataset,
    column_spec_display_name=split_column_name,
)

___

## 5. Creating a model

### Train a model

Specify the duration of the training. For example, `train_budget_milli_node_hours=1000` runs the training for one hour. 

If your Colab times out, use `client.list_models()` to check whether your model has been created. Then use model name to continue to the next steps. Run the following command to retrieve your model.

```python
    model = client.get_model(model_display_name=model_display_name) 
```

In [ ]:
#@title Create model { vertical-output: true }

model_display_name = 'energy_model' #@param {type:'string'}
model_train_hours = 12 #@param {type:'integer'}
model_optimization_objective = 'MINIMIZE_MAE' #@param {type:'string'}

create_model_response = client.create_model(
    model_display_name,
    dataset=dataset,
    optimization_objective=model_optimization_objective,
    train_budget_milli_node_hours=model_train_hours * 1000,
)

print('Dataset import operation: {}'.format(create_model_response.operation))
# Wait until model training is done.
model = create_model_response.result()
model_name = model.name
model

In [ ]:
#@title Model Metrics {vertical-output: true }

metrics= [x for x in client.list_model_evaluations(model=model)][-1]
metrics.regression_evaluation_metrics

![alt text](https://storage.googleapis.com/images_public/automl_test.png)

In [ ]:
#@title Feature Importance {vertical-output: true }

feat_list = [(x.feature_importance, x.column_display_name) for x in model.tables_model_metadata.tables_model_column_info]
feat_list.sort(reverse=True)
feat_list[:15]

![alt text](https://storage.googleapis.com/images_public/feature_importance.png)
![alt text](https://storage.googleapis.com/images_public/loc_portugal.png)
![alt text](https://storage.googleapis.com/images_public/weather_schema.png)
![alt text](https://storage.googleapis.com/images_public/training_schema.png)

___